
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


<strong>Imports :

In [1]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

from vision_final import *

import cv2


In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [3]:
await tdmclient.notebook.stop()

## Async implementation:

In [2]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 0102d6c0-0c6c-479f-8406-474092b0daba

In [3]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [-10787],
 '_imot': [0, 0],
 '_integrator': [0, 0],
 '_productId': [8],
 '_vbat': [793, 795],
 'acc': [0, 0, 20],
 'acc._tap': [90],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10525, 8956, 13552, 13521, 14834],
 'buttons._noise': [36, 28, 41, 40, 41],
 'buttons._raw': [10531, 8955, 13562, 13521, 14832],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-10787],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [256],
 'mic.intensity': [2],
 'mic.threshold': [0],
 'motor.left.pwm': [0],
 'motor.left.speed': [0],
 'motor.left.target': [0],
 'motor.right.pwm': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.rx'

In [2]:

def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta

def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

In [3]:
def path(positions, initial_orientation):
    nb_node = 3
    rotations = np.zeros(positions.shape[0]-1)
    distances =  np.zeros(positions.shape[0]-1)
    
    rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
    orientation = initial_orientation+rotations[0]
    
    for i in range(1,positions.shape[0]-1):
        rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
        distances[i-1] = distance_to_cover(positions[i-1],positions[i])
        orientation = orientation+rotations[i]

    distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])
    rotation_ratios = rotations/360
    return distances, rotations, orientation, rotation_ratios


<strong>Kalman filter : 

In [4]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [5]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

<strong> Variance for position :

In [6]:
qpx = 0.04 # variance on position state
qpy = 0.04
rpx = 0.25 # variance on position measurement 
rpy = 0.25

qpx = 0.04 # variance on position state
qpy = 0.04 
rpx = 0.25 # variance on position measurement 
rpy = 0.25


<strong>Variance for angle : 

In [7]:
std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = std_speed/2 # variance on speed measurement 

print(q_theta_nu,r_theta_nu)

5.885 5.885


In [8]:
# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor_straight = 0.338; #straight
speed_conv_factor_left = 0.403;
speed_conv_factor_right = 0.425;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    if abs(Vr-Vl) < 30:
        speed_conv_factor = speed_conv_factor_straight
    elif Vr<Vl:
        speed_conv_factor = speed_conv_factor_right
    else:
        speed_conv_factor = speed_conv_factor_left
    
    
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);
  
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est

In [9]:
import time

In [10]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [11]:
import asyncio
import time
    
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

In [12]:
def filter():#x_est, P_est, Ts):
    global x_est, P_est, Ts, read, angle, Vr, Vl, ret, frame

    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]}) 

    #print(thymio_data[-1]["left_speed"])
    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]
    #print(Vr, Vl)

    if ret==1:
        thymio_result = locate_thymio(frame)
        thymio_loc = thymio_result[0]
        camera_x = thymio_result[0][0]
        camera_y = thymio_result[0][1]
        camera_angle = thymio_result[1]*360/(2*np.pi)
        ret = 0 
    
    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
    #print(x_est)
    

## Global navigation

In [13]:
from globalNavigation import Point, Edge, Graph, build_graph, Astar 

In [14]:
def global_navigation(polys, start, goal):

    graph = Graph(polys)
    
    nodes, closed = Astar(graph, start, goal)
    
    #for node in nodes:
    nodes = nodes[::-1]
    coordinates = np.array(nodes)
    
    points = np.zeros((len(coordinates),2))
    
    for i in range(len(coordinates)):
        points[i]= [coordinates[i].x,coordinates[i].y]
        
    return points
'''Change the Scale !!!!'''

'Change the Scale !!!!'

## Local avoidance

In [15]:
def test_saw_obstacle(threshold, verbose=True):
    print("\t Test saw obstacle: ")
    if any([x > threshold for x in node['prox.horizontal'][0:5]]):
        if verbose: print("\t\t Obstacle detected")
        return True
    
    return False

In [16]:
async def path_following(motor_speed=100, threshold=0,  verbose=False):
    
    global n, positions, x_est, P_est, Ts, read, Vr, Vl, n

    saw_obstacle = False
    
    if verbose: print("\t Path following ")
    await node.set_variables(motors(motor_speed, motor_speed))
           
    prev_state="local"
    
    while not saw_obstacle:
        
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state=="local": 
                prev_state="global"
            saw_obstacle = True
            await client.sleep(0.1) #otherwise, variables would not be updated
        else:
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
            #print(x_est)
            stop = time.time()
            #print("time elapsed : ", start - stop)
            
    return 

In [17]:
async def local_avoidance(motor_speed=100, threshold=0, verbose=False):
    obstacleSpeedGain = [6, 4, -2, -6, -8]
    saw_obstacle = True
    speedL = motor_speed
    speedR = motor_speed
    prev_state = "global"

    if verbose:
        print("Local avoidance")

    while saw_obstacle:
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state == "global":
                prev_state = "local"
                print("\t Transitioning to local avoidance")

            for i in range(5):
                speedL += node["prox.horizontal"][i] * obstacleSpeedGain[i] // 100
                speedR += node["prox.horizontal"][i] * obstacleSpeedGain[4 - i] // 100

          
            await node.set_variables(motors(speedL, speedR))
            if verbose:
                print("\t\t Adjusting for obstacle")

        else:
            saw_obstacle = False
            if prev_state == "local":
                #prev_state = "global"
                print(prev_state)
                print("\t Transitioning to global navigation")
                
               

            # Reset motor speeds when no obstacles are detected
            speedL = motor_speed
            speedR = motor_speed
            await node.set_variables(motors(speedL, speedR))
            
            # Add a small delay to avoid continuously checking for obstacles too quickly
            await client.sleep(0.1)




In [18]:
# thymio_data = []
# initial_orientation = 0 #degree

# # Calculation of the fastest path # To get from the vision
# polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
# start = Point(76, 81)
# goal = Point(498, 762)

# positions = global_navigation(polys, start, goal)

# positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
# angle_threshold = 5 #degrees
# x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
# P_est = 0
# Ts = 0.1
# count = 0
# read = 1
# Vr = 0
# Vl = 0
# position_threshold = 10

# n = 1
# await node.wait_for_variables() # wait for Thymio variables values
# rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
# try:
#     print("entered try")
#     for i in range(10):
#         await path_following(verbose=True)
#         await local_avoidance(verbose=True)
#     #node.send_set_variables(motors(0, 0))
# finally:
#     rt.stop()
#     node.send_set_variables(motors(0, 0))
    

In [19]:
await node.set_variables(motors(0, 0))

NameError: name 'node' is not defined

## Main

In [21]:
thymio_data = []
initial_orientation = 0 #degree

# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

# Applying vision functions
thymio_result = locate_thymio(frame)
obstacles_result = locate_static_obstacles(frame, 50)
goal_result = locate_goal(frame)  
table_origin_result = locate_table_origin(frame)

start = Point(thymio_result[0][0][0], thymio_result[0][0][1])
goal = Point(goal_result[0], goal_result[1])
table_origin = Point(table_origin_result[0], table_origin_result[1])

# 
polys = []
poly = []
n_polys=0
for polygone in obstacles_result:
    n_polys += 1
    for corner in polygone:
        poly.append(Point(corner[0],corner[1]))
    polys.append(poly)
    poly = []

# Calculation of the fastest path # To get from the vision
positions = global_navigation(polys, start, goal)

# angle_threshold = 5 #degrees
# x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
# P_est = 0
# Ts = 0.1
# count = 0
# read = 1
# Vr = 0
# Vl = 0
# position_threshold = 10

# n = 1
# await node.wait_for_variables() # wait for Thymio variables values
# rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)

# async def main():    
#     global x_est, P_est, Ts, read, Vr, Vl, n
    
#     try:
#         for i in range(0,100):
#             start = time.time()
#             if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
#                 if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
#                     n+=1
#             angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
#             if (abs(angle)<angle_threshold):
#                 Vr = 100
#                 Vl = 100
#                 drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
#                 await drive_task

#             else:
#                 if angle>0:
#                     right = 1
#                     Vr = -100
#                     Vl = 100
#                 else:
#                     right = 0
#                     Vr = 100
#                     Vl = -100

#                 turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
#                 await turn_task
#              #print(x_est)
#             stop = time.time()
#             print("time elapsed : ", start - stop)
#     finally:
#         rt.stop()
#         node.send_set_variables(motors(0, 0))
# await main()

polys [[Point(1135.00, 289.00), Point(1496.00, 289.00), Point(1496.00, 723.00), Point(1135.00, 723.00)], [Point(362.00, -50.00), Point(930.00, -50.00), Point(930.00, 360.00), Point(362.00, 360.00)]]
--------
 nodes  [[1634.23232323  109.68686869]
 [1135.          289.        ]
 [ 378.          711.        ]]
start (1634.23, 109.69) goal (378.00, 711.00)


/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## With vision

In [37]:
thymio_data = []
initial_orientation = 0 #degree

# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

# Applying vision functions
thymio_result = locate_thymio(frame)
obstacles_result = locate_static_obstacles(frame, 50)
goal_result = locate_goal(frame)  
table_origin_result = locate_table_origin(frame)

start = Point(thymio_result[0][0][1], thymio_result[0][0][1])
goal = Point(goal_result[0], goal_result[1])
table_origin = Point(table_origin_result[0], table_origin_result[1])
#polys = obstacles_result[2]

# Calculation of the fastest path # To get from the vision
# Should be calculated directly with the camera feed
# print("---------\n", obstacles_result, "----------\n")
polys = []
poly = []
n_polys=0
for polygone in obstacles_result:
    n_polys += 1
    for corner in polygone:
        poly.append(Point(corner[0],corner[1]))
    polys.append(poly)
    poly = []
    
# print("Polys = ", polys)
# print("npolys = ", n_polys)
#polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
#start = Point(76, 81)
#goal = Point(498, 762)

positions = global_navigation(polys, start, goal) * 0.4
initial_orientation = thymio_result[1]*360/(2*np.pi)

print("--------",positions)



#positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
Vr = 0
Vl = 0
position_threshold = 10
ret = 0

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
rt_2 = RepeatedTimer(Ts, camera_acquisition) #every Ts, get image 


try:
    print("entered try")
    for i in range(10):
        await path_following(verbose=True)
        await local_avoidance(verbose=True)
    #node.send_set_variables(motors(0, 0))
finally:
    rt.stop()
    node.send_set_variables(motors(0, 0))


-------- [[ 43.6  43.6]
 [144.8 144.4]
 [150.8 284.4]]


/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


NameError: name 'camera_acquisition' is not defined

In [ ]:
def camera_acquisition():
    global ret, frame
    
    ret, frame = cap.read()